# Given specific conditions give haloIDs that fulfill them (CAMELS)

In [4]:
import yt
import pyxsim

import h5py
import numpy as np

import os
from regions import RectangleSkyRegion
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy import wcs
from astropy.io import fits

from soxs.utils import soxs_cfg
soxs_cfg.set("soxs", "bkgnd_nH", "0.018") # avoid configparser error by specifying here
import soxs

soxs : [WARNING  ] 2025-05-11 17:16:54,260 Setting 'soxs_data_dir' to /home/jovyan/.cache/soxs for this session. Please update your configuration if you want it somewhere else.


In [9]:
snap = "060"
num = 3

In [10]:
f_snapshot = f'/home/jovyan/Data/Sims/IllustrisTNG/EX/EX_{num}/snapshot_{snap}.hdf5'
f_catalog  = f'/home/jovyan/Data/FOF_Subfind/IllustrisTNG/EX/EX_{num}/groups_{snap}.hdf5'

In [16]:
f = h5py.File(f_catalog, 'r')

c = 299792458  # speed of light in m/s
mp = 1.67 * 1e-27  # kg
G = 6.674 * 1e-11  # N m^2 kg^-2
sigma_t = 6.652 * 1e-29  # m^2
h = 0.6774
e_r = 0.2

Mhalo = f['Group/Group_M_Crit200'][:] * 1e10/ h
M = f['Subhalo/SubhaloBHMass'][:] * 1e10 / h
subID = np.arange(len(Mbh))
phid = f['Subhalo/SubhaloGrNr'][:]
phm = Mhalo[phid[subID]]
Mdot = f['Subhalo/SubhaloBHMdot'][:] * 10.22

group_firstsub = f['Group/GroupFirstSub'][:]
group_firstsub = group_firstsub[group_firstsub != -1]
halo_centralID = np.array(group_firstsub)

central = np.zeros(len(M))
central[halo_centralID] = 1

indices = np.where((M > 1e6) & (phm > 10**10) & (phm < 10**11) & (central == 1))

Mdot_filtered = Mdot[indices]
M_filtered = M[indices]
subID_filtered = subID[indices]

Mdot_edd = (4 * np.pi * G * M_filtered * mp) / (e_r * c * sigma_t) / 3.16887646e-8  # M_sun/yr

edd_ratio = Mdot_filtered / Mdot_edd

efficiency = 10 * edd_ratio * e_r
efficiency[edd_ratio > 0.1] = e_r /(1-e_r)
Lbol = efficiency * Mdot_filtered * (1.989e+30 / 3.154e+7) * c**2 * 1e+7         #ergs/s

threshold = 0.1
Lcut = 1e41

sa_agn = subID_filtered[np.where((edd_ratio > threshold) & (Lbol > Lcut))]
sa_non = subID_filtered[np.where((edd_ratio > threshold) & (Lbol <= Lcut))]
sn_agn = subID_filtered[np.where((edd_ratio <= threshold) & (Lbol > Lcut))]
sn_non = subID_filtered[np.where((edd_ratio <= threshold) & (Lbol <= Lcut))]


ha_agn = phid[sa_agn]
ha_non = phid[sa_non]
hn_agn = phid[sn_agn]
hn_non = phid[sn_non]

np.savez("accretor_agn50_1011.npz", ha_agn = ha_agn, ha_non = ha_non, hn_agn = hn_agn, hn_non = hn_non)

# Print stats
total = len(subID_filtered)
print("Total halos:", total)
print("High accretor Central AGNs:", len(ha_agn))
print("High accretor Central Non-AGNs:", len(ha_non))
print("Non-accretor Central AGNs:", len(hn_agn))
print("Non-accretor Central Non-AGNs:", len(hn_non))

Total halos: 267
High accretor Central AGNs: 3
High accretor Central Non-AGNs: 0
Non-accretor Central AGNs: 96
Non-accretor Central Non-AGNs: 168
